In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import time

from numpy import diag, log, sqrt
from numpy.linalg import eig, inv, norm, svd
from scipy.linalg import expm, solve_lyapunov, solve_sylvester, sqrtm

## multidim. OU process

$$d X_t = AX_t dt + B_1 u_t dt + B_2 d W_t, \qquad X_0 = x, \qquad Y_t = C X_t$$

In [ ]:
d = 50
r = 5

A = - np.diag([1] * r + [0.1] * (d - r)) + 0.005 * np.random.randn(d, d)
B1 = np.diag([1] * r + [0.1] * (d - r)) + 0.005 * np.random.randn(d, d)
B2 = np.diag([1] * r + [0.1] * (d - r)) + 0.005 * np.random.randn(d, d)
B = np.concatenate([B1, B2], 1)
#C = np.diag([1] * r + [0.01] * (d - r)) + 0.001 * np.random.randn(d, d)
C = np.concatenate([np.diag([1] * r), np.zeros([r, d - r])], 1)

In [ ]:
#X_0 = np.ones([d, 1]) * 0
X_0 = np.array([1] * r + [0] * (d - r))[:, np.newaxis]

if ~np.all(np.linalg.eigvals(A) < 0):
    print('not all EV of A are negative')

T_end = 2

def u(x, t):
    return np.sin(t) * np.ones(x.shape)

u_L2 = sqrt(d * 0.5 * (T_end - np.sin(T_end) * np.cos(T_end))) 

B_in = X_0

## Theorem 1 and Corollary 3.4 - reduce homogenous and inhomogenous process separately

In [ ]:
P_inh = solve_lyapunov(A, -B.dot(B.T))
P_hom = solve_lyapunov(A, -B_in.dot(B_in.T))

Q = solve_lyapunov(A.T, -C.T.dot(C))

#K_inh = sqrtm(P_inh) #  this can yield complex numbers due to numerical issues
#L = sqrtm(Q)

V_P_inh, sigmas_P_inh, U_P_inh_T = svd(P_inh)
K_inh = V_P_inh.dot(sqrt(diag(sigmas_P_inh))).dot(U_P_inh_T)

V_Q, sigmas_Q, U_Q_T = svd(Q)
L = V_Q.dot(sqrt(diag(sigmas_Q))).dot(U_Q_T)

V_inh, sigmas_inh, U_inh_T = svd(K_inh.T.dot(L))
Sigma_inh = np.diag(sigmas_inh)
Sigma_inh_inv_05 = np.diag(1 / sqrt(sigmas_inh))

T_inh = Sigma_inh_inv_05.dot(U_inh_T).dot(L.T)
T_inh_inv = K_inh.dot(V_inh).dot(Sigma_inh_inv_05)
A_inh_ = T_inh.dot(A).dot(T_inh_inv)
B_inh_ = T_inh.dot(B)
C_inh_ = C.dot(T_inh_inv)

plt.scatter(range(d), sigmas_inh, s=1)
plt.title('Hankel singular values - inhomogenous');
plt.yscale('log')
plt.ylim(0.8 * min(sigmas_inh), 1.2 * max(sigmas_inh));

In [ ]:
if np.any(X_0) != 0:

    #K_hom = sqrtm(P_hom)
    
    V_P_hom, sigmas_P_hom, U_P_hom_T = svd(P_hom)
    K_hom = V_P_hom.dot(sqrt(diag(sigmas_P_hom))).dot(U_P_hom_T)

    V_hom, sigmas_hom, U_hom_T = svd(K_hom.T.dot(L))
    Sigma_hom = np.diag(sigmas_hom)
    Sigma_hom_inv_05 = np.diag(1 / sqrt(sigmas_hom))

    T_hom = Sigma_hom_inv_05.dot(U_hom_T).dot(L.T)
    T_hom_inv = K_hom.dot(V_hom).dot(Sigma_hom_inv_05)
    A_hom_ = T_hom.dot(A).dot(T_hom_inv)
    C_hom_ = C.dot(T_hom_inv)
    B_in_ = T_hom.dot(B_in)

    plt.scatter(range(d), sigmas_hom, s=1)
    plt.title('Hankel singular values - homogenous');
    plt.yscale('log')
    plt.ylim(0.8 * min(sigmas_hom), 1.2 * max(sigmas_hom));

In [ ]:
L2_bounds = []
sup_bounds = []
simulation_max_sqrt = []
simulation_max = []
simulation_sqrt_L2 = []

for r in range(1, d):
    A_r = A_inh_[:r, :r]
    B_r = B_inh_[:r, :]
    B1_r = B_r[:, :d]
    B2_r = B_r[:, d:]
    C_r = C_inh_[:, :r]
    
    if np.any(X_0) != 0:
        A_0_r = A_hom_[:r, :r]
        C_0_r = C_hom_[:, :r]
        B_in_r = B_in_[:r, :]

    # compute bounds
    if np.any(X_0) != 0:
        P_0 = solve_lyapunov(A, -B_in.dot(B_in.T))
        P_0_g = solve_sylvester(A, A_0_r.T, -B_in.dot(B_in_r.T))
        P_0_r = solve_lyapunov(A_0_r, -B_in_r.dot(B_in_r.T))

    P_r = solve_lyapunov(A_r, -B_r.dot(B_r.T))
    P_g = solve_sylvester(A, A_r.T, -B.dot(B_r.T))
    
    # the absolute value inside the traces is done due to numerical instabilities
    if np.any(X_0) != 0:
        L2_bounds.append(sqrt(2 * T_end) * max(1, u_L2) * sqrt(np.abs(np.trace(C.dot(P_inh).dot(C.T) + C_r.dot(P_r).dot(C_r.T)
                                                   - 2 * C.dot(P_g).dot(C_r.T))))
                 + sqrt(np.abs(np.trace(C.dot(P_0).dot(C.T) + C_0_r.dot(P_0_r).dot(C_0_r.T) - 2 * C.dot(P_0_g).dot(C_0_r.T)))))
    else:
        L2_bounds.append(sqrt(2 * T_end) * max(1, u_L2) * sqrt(np.trace(C.dot(P_inh).dot(C.T) + C_r.dot(P_r).dot(C_r.T)
                                           - 2 * C.dot(P_g).dot(C_r.T))))
        sup_bounds.append(sqrt(2) * max(1, u_L2) * sqrt(np.abs(np.trace(C.dot(P_inh).dot(C.T) + C_r.dot(P_r).dot(C_r.T) - 2 * C.dot(P_g).dot(C_r.T)))))

    
    # run simulation
    delta_t = 0.01
    N = int(np.ceil(T_end / delta_t))
    M = 100

    X = np.zeros([M, N + 1, d])
    X_r = np.zeros([M, N + 1, r])
    
    X[:, 0] = np.zeros(d) # X_0[:, 0]
    X_r[:, 0] = np.zeros(r)

    for n in range(N):
        xi = np.random.randn(d, M)
        u_t = u(X[:, n, :], n * delta_t).T
        X[:, n + 1, :] = (X[:, n, :] + A.dot(X[:, n, :].T).T * delta_t
                          + B1.dot(u_t).T * delta_t + B2.dot(xi).T * sqrt(delta_t))
        X_r[:, n + 1, :] = (X_r[:, n, :] + A_r.dot(X_r[:, n, :].T).T * delta_t
                            + B1_r.dot(u_t).T * delta_t + B2_r.dot(xi).T * sqrt(delta_t))

    Y = X.dot(C.T)  
    Y_r = X_r.dot(C_r.T)

    if np.any(X_0) != 0:
        Y = Y + np.repeat(np.array([C.dot(expm(A * n * delta_t)).dot(B_in)[:, 0] for n in range(N + 1)])[np.newaxis, :, :], M, 0)
        Y_r = Y_r + np.repeat(np.array([C_0_r.dot(expm(A_0_r * n * delta_t)).dot(B_in_r)[:, 0] for n in range(N + 1)])[np.newaxis, :, :], M, 0)

        
    simulation_sqrt_L2.append(sqrt(np.mean(np.sum(np.sum((Y - Y_r)**2, 2), 1) * delta_t)))

    if np.all(X_0) == 0:
        simulation_max_sqrt.append(np.max(sqrt(np.mean(np.sum((Y - Y_r)**2, 2), 0))))
        simulation_max.append(np.max(np.mean(sqrt(np.sum((Y - Y_r)**2, 2)), 0)))

## Theorem 2

In [ ]:
if np.all(X_0 == 0):
    P = solve_lyapunov(A, -B.dot(B.T))
else:
    P = solve_lyapunov(A, -B1.dot(B1.T) + A.dot(B_in.dot(B_in.T) + B2.dot(B2.T)) + (B_in.dot(B_in.T) + B2.dot(B2.T)).dot(A.T))
Q = solve_lyapunov(A.T, -C.T.dot(C))

#K = sqrtm(P)
#L = sqrtm(Q)

V_P, sigmas_P, U_P_T = svd(P)
K = V_P.dot(sqrt(diag(sigmas_P))).dot(U_P_T)

V_Q, sigmas_Q, U_Q_T = svd(Q)
L = V_Q.dot(sqrt(diag(sigmas_Q))).dot(U_Q_T)

V, sigmas, U_T = svd(K.T.dot(L))
Sigma = np.diag(sigmas)
Sigma_inv_05 = np.diag(1 / sqrt(sigmas))

T = Sigma_inv_05.dot(U_T).dot(L.T)
T_inv = K.dot(V).dot(Sigma_inv_05)
A_ = T.dot(A).dot(T_inv)
B_ = T.dot(B)
C_ = C.dot(T_inv)

plt.scatter(range(d), sigmas, s=1)
plt.title('Hankel singular values');
plt.yscale('log')
plt.ylim(0.8 * min(sigmas), 1.2 * max(sigmas));

In [ ]:
L2_bounds_2 = []
L2_bounds_Hankel = []
simulation_sqrt_L2_2 = []

for r in range(1, d):
    
    A_r = A_[:r, :r]
    B_r = B_[:r, :]
    C_r = C_[:, :r]

    P_r = solve_lyapunov(A_r, -B_r.dot(B_r.T))
    P_g = solve_sylvester(A, A_r.T, -B.dot(B_r.T))
    Sigma_2 = np.diag(sigmas[r:])
    P_g1 = T.dot(P_g)[:r, :]
    P_g2 = T.dot(P_g)[r:, :]
    
    A_12 = A_[:r, r:]
    A_21 = A_[r:, :r]
    A_22 = A_[r:, r:]
    B_2 = B_[r:, :]
    
    B1_r = B_r[:, :d]
    B2_r = B_r[:, d:]
    
    A_hat = np.zeros([d + r, d + r])
    A_hat[0:d, 0:d] = A
    A_hat[d:d + r, d:d + r] = A_r
    B1_hat = np.concatenate([B1, B1_r])
    B2_hat = np.concatenate([B2, B2_r])
    B_in_hat = np.concatenate([B_in, T.dot(B_in)[:r, :]])
    C_hat = np.concatenate([C, -C_r], axis=1)

    P_hat = solve_lyapunov(A_hat, -B1_hat.dot(B1_hat.T) + A_hat.dot(B_in_hat.dot(B_in_hat.T) + B2_hat.dot(B2_hat.T))
                                 + (B_in_hat.dot(B_in_hat.T) + B2_hat.dot(B2_hat.T)).dot(A_hat.T))
    Q_hat = solve_lyapunov(A_hat.T, -C_hat.T.dot(C_hat))

    #K_hat = sqrtm(P_hat)
    #L_hat = sqrtm(Q_hat)
    
    V_P_hat, sigmas_P_hat, U_P_hat_T = svd(P_hat)
    K_hat = V_P_hat.dot(sqrt(diag(sigmas_P_hat))).dot(U_P_hat_T)

    V_Q_hat, sigmas_Q_hat, U_Q_hat_T = svd(Q_hat)
    L_hat = V_Q_hat.dot(sqrt(diag(sigmas_Q_hat))).dot(U_Q_hat_T)

    V_hat, sigmas_hat, U_hat_T = svd(K_hat.T.dot(L_hat))
    Sigma_hat = np.diag(sigmas_hat)
    Sigma_hat_inv_05 = np.diag(1 / sqrt(sigmas_hat))

    # compute bounds
    L2_bounds_Hankel.append(sum(sigmas[r:]) * (sqrt(T_end) + sqrt(T_end * np.sum(X_0**2)) + 2 * u_L2))
    L2_bounds_2.append(sum(sigmas_hat) * (sqrt(T_end) + sqrt(T_end * np.sum(X_0**2)) + 2 * u_L2))
    
    # run simulation
    delta_t = 0.01
    N = int(np.ceil(T_end / delta_t))
    M = 100

    X = np.zeros([M, N + 1, d])
    X_r = np.zeros([M, N + 1, r])
    
    X[:, 0] = X_0[:, 0]
    X_r[:, 0] = T.dot(X_0)[:r, 0]

    for n in range(N):
        xi = np.random.randn(d, M)
        u_t = u(X[:, n, :], n * delta_t).T
        X[:, n + 1, :] = (X[:, n, :] + A.dot(X[:, n, :].T).T * delta_t
                          + B1.dot(u_t).T * delta_t + B2.dot(xi).T * sqrt(delta_t))
        X_r[:, n + 1, :] = (X_r[:, n, :] + A_r.dot(X_r[:, n, :].T).T * delta_t
                            + B1_r.dot(u_t).T * delta_t + B2_r.dot(xi).T * sqrt(delta_t))

    Y = X.dot(C.T)  
    Y_r = X_r.dot(C_r.T)
    
    simulation_sqrt_L2_2.append(sqrt(np.mean(np.sum(np.sum((Y - Y_r)**2, 2), 1) * delta_t)))

In [ ]:
I = 3 if np.all(X_0 == 0) == True else 2

fig, ax = plt.subplots(1, I, figsize=(I * 5, 4))
fig.suptitle(r'Ornstein-Uhlenbeck, $d = %d, T = %.0f$, non-zero initial value' % (d, T_end))

i = 0

if np.all(X_0 == 0) == True:
    ax[i].set_title('supremum error')
    ax[i].scatter(range(1, d), np.array(sup_bounds), marker='x', label='bound Theorem 1', color='royalblue')
    ax[i].scatter(range(1, d), np.array(simulation_max_sqrt), marker='x', label=r'simulation Theorem 1', color='green') # $\max_t \sqrt{\mathbb{E}[ \|Y_t - Y_t^r\|^2]}$
    #ax[i].scatter(range(1, d), np.array(simulation_max), marker='.', label=r'simulation, $\max_t \mathbb{E}[ \|Y_t - Y_t^r\|]$')
    ax[i].legend()
    ax[i].set_yscale('log')
    ax[i].set_ylim(0.8 * min(simulation_max_sqrt), 1.2 * max(sup_bounds))
    i += 1

ax[i].set_title(r'$L^2$ error')
ax[i].scatter(range(1, d), np.array(L2_bounds), marker='x', label=r'$L^2$ bound, Corollary 3.4', color='royalblue')
ax[i].scatter(range(1, d), np.array(simulation_sqrt_L2), marker='x', label=r'simulation, Corollary 3.4', color='green') # $\|Y_t - Y_t^r\|_{L^2_{\Omega_T}}$
ax[i].scatter(range(1, d), np.array(L2_bounds_2), marker='.', label=r'$L^2$ bound, Theorem 2', color='orange')
ax[i].scatter(range(1, d), np.array(simulation_sqrt_L2_2), marker='.', label=r'simulation, Theorem 2', color='red')
ax[i].legend(loc='upper right')
ax[i].set_yscale('log')
ax[i].set_ylim(0.8 * min(simulation_sqrt_L2_2), 1.2 * max(L2_bounds_2));

i += 1

ax[i].set_title('Hankel singular values')
if np.all(X_0 == 0) == False:
    ax[i].scatter(range(d), sigmas_inh, marker='x', label='Corollary 3.4, inhom. part', color='royalblue');
    ax[i].scatter(range(d), sigmas_hom, marker='x', label='Corollary 3.4, hom. part', color='green');
    ax[i].set_ylim(0.8 * min(sigmas_hom), 1.2 * max(sigmas))
else:
    ax[i].scatter(range(d), sigmas_inh, marker='x', label='Corollary 3.4, inhom. part', color='royalblue');
    ax[i].set_ylim(0.8 * min(sigmas_inh), 1.2 * max(sigmas))
ax[i].scatter(range(d), sigmas, marker='.', label='Theorem 2', color='orange');
ax[i].set_yscale('log');
ax[i].legend();

fig.tight_layout(rect=[0, 0.03, 1, 0.95])

#fig.savefig('img/BT_OU_d50_zero.pdf') 